In [1]:
import pandas as pd
import numpy as np
from scipy import optimize
from scipy.stats import norm

In [2]:
def getStrikefromDelta(fwd, T, cp, sigma, delta):
    '''
    Inputs :
    fwd: forward spot for time T, i.e. E[S(T)]
    T: time to expiry of the option
    cp: 1 for call , -1 for put
    sigma : implied Black volatility of the option
    delta : delta in absolute value (e.g. 0.25)
    Output :
    K: strike of the option
    
    '''
    vol = sigma*np.sqrt(T/365)
    K = np.zeros(len(cp))
    for i in range(len(cp)):
        if cp[i]==1:
            func = lambda K: norm.cdf(np.log(fwd/K)/vol[i]+0.5*vol[i])-delta[i]
            f_prime = lambda K: norm.pdf(np.log(fwd/K)/vol[i] + 0.5*vol[i])/(-K*vol[i])
        else:
            func = lambda K: norm.cdf(-np.log(fwd/K)/vol[i]-0.5*vol[i])-delta[i]
            f_prime = lambda K: norm.pdf(-np.log(fwd/K)/vol[i] - 0.5*vol[i])/(K*vol[i])
        K[i] = fwd
        K[i] = optimize.newton(func, K[i], fprime=f_prime)
    return K

In [3]:
%run ForwardSpot.ipynb
%run getMarket.ipynb

In [9]:
T = np.array([181])
fwd = getFwdSpot(curve, T)
fwd

array([1.49608694])

In [10]:
Vs = vols.loc[6]
delta = deltas
cp = cps
Vs
cps
delta

[0.1, 0.25, 0.5, 0.25, 0.1]

In [11]:
Ks = getStrikefromDelta(fwd, T, cps, Vs, delta)
Ks

array([1.20318711, 1.34814634, 1.51945174, 1.7161674 , 1.94369918])

In [12]:
%run getBlackCall.ipynb

In [13]:
getBlackCall(fwd, T, Ks, Vs)

array([0.30692656, 0.19144946, 0.09448499, 0.03673301, 0.01247471])